In [1]:
# import multiprocessing as mp
# mp.set_start_method("spawn")
# CCL openmp conflicts with pymc3 parallelization
#####
## %env OMP_NUM_THREADS=1
import pymc3 as pm
import pyccl as ccl
import theano.tensor as tt
from classy import Class
from matplotlib import pyplot as plt
import numpy as np

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Example 1 from Getting Started guide

In [5]:
# Example 1 from https://docs.pymc.io/notebooks/getting_started.html#Sampling-methods

# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1
beta = [1, 2.5]

# Size of dataset
size = 100

# Predictor variable
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(size)*sigma



basic_model = pm.Model()

with basic_model:

    # Priors for unknown model parameters
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta = pm.Normal('beta', mu=0, sigma=10, shape=2)
    sigma = pm.HalfNormal('sigma', sigma=10)

    # Expected value of outcome
    mu = alpha + beta[0]*X1 + beta[1]*X2

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=Y)
    
    # Sample
    step = pm.NUTS()
    trace = pm.sample(5000, step=step)
    
#pm.traceplot(trace, compact=False);
#pm.plot_posterior(trace)
#pm.save_trace(trace, 'pymc3-test-save_trace')

/home/jaimerz/.local/lib/python3.8/site-packages/pymc3/sampling.py:466: FutureWarning: In an upcoming release, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  warnings.warn(
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta, alpha]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 4 seconds.


In [6]:
pm.gelman_rubin(trace)

AttributeError: module 'pymc3' has no attribute 'gelman_rubin'

In [7]:
pm.summary(trace).round(2)

AttributeError: module 'pymc3' has no attribute 'summary'

In [8]:
with basic_model:
    cov = pm.trace_cov(trace)
print(cov)

[[ 1.03495844e-02 -3.27375654e-04  1.41368366e-04 -1.02831148e-04]
 [-3.27375654e-04  8.02366808e-03  1.08077790e-03  1.63941567e-04]
 [ 1.41368366e-04  1.08077790e-03  2.66411787e-01  1.35004994e-04]
 [-1.02831148e-04  1.63941567e-04  1.35004994e-04  5.32806541e-03]]


# Example of defining your own likelihood

In [ ]:
class Lkl(pm.Continuous):
    def __init__(self, mu, *args, **kwargs):
        super(Lkl, self).__init__(*args, **kwargs)
        self.mu = mu
        self.mode = mu

    def logp(self, value):
        mu = self.mu
        return beta_logp(value - mu)


def beta_logp(value):
    return -1.5 * np.log(1 + (value)**2)


with pm.Model() as model:
    beta = Lkl('slope', mu=3, testval=0)
    gaus = pm.Normal('gaus', mu=0, sigma=1)
    step = pm.NUTS()
    trace = pm.sample(2000, tune=1000, init=None, step=step, cores=2)
    
pm.traceplot(trace)
pm.summary(trace).round(2)

#################

with pm.Model() as model:
    beta = Lkl('slope', mu=3, testval=0)
    gaus = pm.Normal('gaus', mu=0, sigma=1)
    step = pm.Metropolis()
    trace = pm.sample(2000, tune=1000, init=None, step=step, cores=2)
    
pm.traceplot(trace)
pm.summary(trace).round(2)

# Blackbox (external distributions that cannot use theano operators)

In [ ]:
# define a theano Op for our likelihood function
class bao_lkl(tt.Op):

    """
    Specify what type of object will be passed and returned to the Op when it is
    called. In our case we will be passing it a vector of values (the parameters
    that define our model) and returning a single "scalar" value (the
    log-likelihood)
    """
    itypes = [tt.dvector] # expects a vector of parameter values when called
    otypes = [tt.dscalar] # outputs a single scalar value (the log likelihood)

    def __init__(self):
        """
        Initialise the Op with various things that our log-likelihood function
        requires. Below are the things that are needed in this particular
        example.

        Parameters
        ----------
        """
            
        # Data from montepython_public/data/COMBINEDDR12_BAO_consensus_dM_Hz
        self.rsfid = 147.78
        cov = np.array([
            [624.707, 23.729, 325.332, 8.34963, 157.386, 3.57778],
            [23.729, 5.60873, 11.6429, 2.33996, 6.39263, 0.968056],
            [325.332, 11.6429, 905.777, 29.3392, 515.271, 14.1013],
            [8.34963, 2.33996, 29.3392, 5.42327, 16.1422, 2.85334],
            [157.386, 6.39263, 515.271, 16.1422, 1375.12, 40.4327],
            [3.57778, 0.968056, 14.1013, 2.85334, 40.4327, 6.25936]
        ])
        self.icov = np.linalg.inv(cov)
        # BAO-only consensus results, Alam et al. 2016
        self.z = np.array([0.38, 0.51, 0.61])
        self.a = 1/(1+self.z)
        # Vectors are multiplied by (rsfid/rs) so that: dM = dM*(rsfid/rs) 
        # and Hz = Hz*(rs/rsfid)
        dM = np.array([1512.39, 1975.22, 2306.68])
        Hz = np.array([81.2087, 90.9029, 98.9647])

        data_vector = np.empty((dM.size + Hz.size), dtype=dM.dtype)
        data_vector[0::2] = dM
        data_vector[1::2] = Hz
        
        self.data = data_vector
        self.model = Class()

    def likelihood(self, theta):
        if np.any(theta < 0):
            return -np.inf

        Omega_c, h = theta
#         cosmo = ccl.Cosmology(Omega_c=Omega_c, Omega_b=0.045, h=h, sigma8=0.78, n_s=0.96,
#                                transfer_function='boltzmann_class')
#         Hz = ccl.background.h_over_h0(cosmo, self.a) * h * 100
#         dM = ccl.background.comoving_angular_distance(cosmo, self.a)
        
#         params = {
#         "h": cosmo["h"],
#         "Omega_cdm": cosmo["Omega_c"],
#         "Omega_b": cosmo["Omega_b"],
#         "Omega_k": cosmo["Omega_k"],
#         "n_s": cosmo["n_s"],
#         "T_cmb": cosmo['T_CMB']}
    
        params = {'h': h,
                  'Omega_cdm': Omega_c}
        
        model = self.model
        model.set(params)
        try:
            model.compute()
        except:
            model.struct_cleanup()
            return -np.inf
        rs = model.rs_drag()
        dM = np.array([model.angular_distance(zi) * (1. + zi) for zi in self.z])
        Hz = np.array([model.Hubble(zi) for zi in self.z]) * 2.99792458e8 / 1000.0
        model.struct_cleanup()
        model.empty()
                
        # Not sure how to compute r_s in CCL. Not included
        th_vector = np.empty((dM.size + Hz.size), dtype=dM.dtype)
        th_vector[0::2] = dM * self.rsfid / rs
        th_vector[1::2] = Hz * rs / self.rsfid
        
        logl = -0.5 * (th_vector - self.data).dot(self.icov).dot(th_vector - self.data)
        return logl

    def perform(self, node, inputs, outputs):
        # the method that is used when calling the Op
        theta, = inputs  # this will contain my variables

        # call the log-likelihood function
        logl = self.likelihood(theta)

        outputs[0][0] = np.array(logl) # output the log-likelihood
    

logl = bao_lkl()
    
with pm.Model():
    # uniform priors on m and c
    Omega_c = pm.Normal('Omega_c', mu=0.26, sigma=0.05)
    h = pm.Normal('h', mu=0.68, sigma=0.05)

    # convert m and c to a tensor vector
    theta = tt.as_tensor_variable([Omega_c, h])

    # use a DensityDist (use a lamdba function to "call" the Op)
    pm.DensityDist('likelihood', lambda v: logl(v), observed={'v': theta})

    step = pm.Metropolis()
    trace = pm.sample(100, step=step, cores=2)

# plot the traces
_ = pm.traceplot(trace)

In [ ]:
pm.summary(trace).round(2)

In [ ]:
pm.plot_posterior(trace)

# Blackbox 2: power spectra likelihood

In [ ]:
# Compute the true power spectra for a fiducial cosmology

h = 0.67
Omega_c = 0.26

def compute_cl(h, Omega_c, z, pz, bz, ells):

    cosmo = ccl.Cosmology(Omega_c=Omega_c, Omega_b=0.045, h=h, sigma8=0.78, n_s=0.96,
                          transfer_function='boltzmann_class')
    # Get tracer
    tracer1 = ccl.NumberCountsTracer(cosmo,has_rsd=False,dndz=(z,pz),bias=(z,bz))
    cls = ccl.angular_cl(cosmo, tracer1, tracer1, ells)

    return cls


# We don't add noise, yet

class lkl(tt.Op):

    """
    Specify what type of object will be passed and returned to the Op when it is
    called. In our case we will be passing it a vector of values (the parameters
    that define our model) and returning a single "scalar" value (the
    log-likelihood)
    """
    itypes = [tt.dvector] # expects a vector of parameter values when called
    otypes = [tt.dscalar] # outputs a single scalar value (the log likelihood)

    def __init__(self):
        """
        Initialise the Op with various things that our log-likelihood function
        requires. Below are the things that are needed in this particular
        example.

        Parameters
        ----------
        """
        h = 0.67
        Omega_c = 0.26
        
        
        # Load effective ells
        fname = 'effective_ell_namaster.txt'
        self.ells = np.loadtxt(fname)
        # Import z, pz
        fname = 'des_clustering/dndz_bin0.txt'
        self.z, self.pz = np.loadtxt(fname, usecols=(1,3), unpack=True)
        # Calculate bias
        bias = 1.41
        self.bz = bias*np.ones(self.z.shape)
        
        self.data = compute_cl(h, Omega_c, self.z, self.pz, self.bz, self.ells)
        cov = np.load('cov_DESgc0_DESgc0_DESgc0_DESgc0.npz')['arr_0']
        self.icov = np.linalg.inv(cov)
            
        print('Likelihoood initialized')

    def likelihood(self, theta):
        if np.any(theta < 0):
            return -np.inf

        Omega_c, h = theta

        if (h < 0.6) or (h > 0.8):
            return -np.inf
        elif (Omega_c < 0.2) or (Omega_c > 0.35):
            return -np.inf
        
        try:
            cosmo = ccl.Cosmology(Omega_c=Omega_c, Omega_b=0.045, h=h, sigma8=0.78, n_s=0.96,
                                  transfer_function='boltzmann_class')
#             cosmo.compute_nonlin_power()
#             sigma = ccl.background.h_over_h0(cosmo, 1) / 0.67 - 1
#             th_vector = self.data + np.random.normal(scale=np.abs(sigma), size=self.data.size)
            
            tracer1 = ccl.NumberCountsTracer(cosmo,has_rsd=False,dndz=(self.z,self.pz),bias=(self.z,self.bz))
            th_vector = ccl.angular_cl(cosmo, tracer1, tracer1, self.ells)
        except Exception as e:
            return -np.inf
        
        
        logl = -0.5 * (th_vector - self.data).dot(self.icov).dot(th_vector - self.data)
        return logl

    def perform(self, node, inputs, outputs):
        # the method that is used when calling the Op
        theta, = inputs  # this will contain my variables

        # call the log-likelihood function
        logl = self.likelihood(theta)

        outputs[0][0] = np.array(logl) # output the log-likelihood


logl = lkl()

with pm.Model():
    # uniform priors on m and c
    Omega_c = pm.Normal('Omega_c', mu=0.26, sigma=0.005)
    h = pm.Normal('h', mu=0.68, sigma=0.005)

    # convert m and c to a tensor vector
    theta = tt.as_tensor_variable([Omega_c, h])

    # use a DensityDist (use a lamdba function to "call" the Op)
    pm.DensityDist('likelihood', lambda v: logl(v), observed={'v': theta})

    step = pm.Metropolis()
    trace = pm.sample(100, step=step, cores=2)

# plot the traces
_ = pm.traceplot(trace)

# Blackbox 3: comparing NUTS vs MH

Following https://docs.pymc.io/notebooks/blackbox_external_likelihood.html we want to build a likelihood for a gc-gc power spectrum, using DES covmat, redshift bins, etc. as in the MP likelihood.

We want to check if NUTS with the gradient function is faster or not than MH.

In [2]:
# Define our likelihood and functions to load observational data and compute the Cl.
import numpy as np
import time
from classy import CosmoComputationError

def compute_cl(theta, data):
    h = theta[0]
    Omega_c = theta[1]
    z = data['z']
    bz = data['bz']
    pz = data['pz']
    ells = data['ells']
    
    cosmo = ccl.Cosmology(Omega_c=Omega_c, Omega_b=0.045, h=h, sigma8=0.78, n_s=0.96,
                          transfer_function='boltzmann_class')
    # Get tracer
    tracer1 = ccl.NumberCountsTracer(cosmo,has_rsd=False,dndz=(z,pz),bias=(z,bz))
    cls = ccl.angular_cl(cosmo, tracer1, tracer1, ells)

    return cls

def cl_lkl(theta, data):
    start = time.time()
    try:
        model = compute_cl(theta, data)
    except CosmoComputationError:
        print('Failed likelihood')
        return -np.inf
    icov = data['icov']
    loglkl = -0.5*(model - data['cls']).dot(icov).dot(model - data['cls'])
    end = time.time()
    print('Computed likelihood in:', end - start, 's')
    return loglkl

def load_data():
        # Load effective ells
        fname = 'effective_ell_namaster.txt'
        ells = np.loadtxt(fname)
        # Import z, pz
        fname = 'des_clustering/dndz_bin0.txt'
        z, pz = np.loadtxt(fname, usecols=(1,3), unpack=True)
        # Calculate bias
        bias = 1.41
        bz = bias*np.ones(z.shape)
        
        cov = np.load('cov_DESgc0_DESgc0_DESgc0_DESgc0.npz')['arr_0']
        icov = np.linalg.inv(cov)
                
        cls = np.zeros(ells.size)  # Here we would load the cls gc0-gc0 file
        data = {'ells': ells, 'cls': cls, 'z': z, 'pz': pz,
                'bz': bz, 'icov': icov}
        theta_fid = [0.67, 0.26]
        cls = compute_cl(theta_fid, data)
        data['cls'] = cls

        return data

theta_fid = [0.67, 0.26]  # [h, Omega_c]
data = load_data()
chi2 = cl_lkl([0.67, 0.26], data)
print(chi2) 

OSError: effective_ell_namaster.txt not found.

In [ ]:
%load_ext Cython

In [ ]:
%%cython
# Define the gradient function. Copied from:
# https://docs.pymc.io/notebooks/blackbox_external_likelihood.html


import cython
cimport cython

import numpy as np
cimport numpy as np

import warnings

def gradients(vals, func, releps=1e-3, abseps=None, mineps=1e-9, reltol=1e-3,
              epsscale=0.5):
    """
    Calculate the partial derivatives of a function at a set of values. The
    derivatives are calculated using the central difference, using an iterative
    method to check that the values converge as step size decreases.

    Parameters
    ----------
    vals: array_like
        A set of values, that are passed to a function, at which to calculate
        the gradient of that function
    func:
        A function that takes in an array of values.
    releps: float, array_like, 1e-3
        The initial relative step size for calculating the derivative.
    abseps: float, array_like, None
        The initial absolute step size for calculating the derivative.
        This overrides `releps` if set.
        `releps` is set then that is used.
    mineps: float, 1e-9
        The minimum relative step size at which to stop iterations if no
        convergence is achieved.
    epsscale: float, 0.5
        The factor by which releps if scaled in each iteration.

    Returns
    -------
    grads: array_like
        An array of gradients for each non-fixed value.
    """

    grads = np.zeros(len(vals))

    # maximum number of times the gradient can change sign
    flipflopmax = 10.

    # set steps
    if abseps is None:
        if isinstance(releps, float):
            eps = np.abs(vals)*releps
            eps[eps == 0.] = releps  # if any values are zero set eps to releps
            teps = releps*np.ones(len(vals))
        elif isinstance(releps, (list, np.ndarray)):
            if len(releps) != len(vals):
                raise ValueError("Problem with input relative step sizes")
            eps = np.multiply(np.abs(vals), releps)
            eps[eps == 0.] = np.array(releps)[eps == 0.]
            teps = releps
        else:
            raise RuntimeError("Relative step sizes are not a recognised type!")
    else:
        if isinstance(abseps, float):
            eps = abseps*np.ones(len(vals))
        elif isinstance(abseps, (list, np.ndarray)):
            if len(abseps) != len(vals):
                raise ValueError("Problem with input absolute step sizes")
            eps = np.array(abseps)
        else:
            raise RuntimeError("Absolute step sizes are not a recognised type!")
        teps = eps

    # for each value in vals calculate the gradient
    count = 0
    for i in range(len(vals)):
        # initial parameter diffs
        leps = eps[i]
        cureps = teps[i]

        flipflop = 0

        # get central finite difference
        fvals = np.copy(vals)
        bvals = np.copy(vals)

        # central difference
        fvals[i] += 0.5*leps  # change forwards distance to half eps
        bvals[i] -= 0.5*leps  # change backwards distance to half eps
        cdiff = (func(fvals)-func(bvals))/leps

        while 1:
            fvals[i] -= 0.5*leps  # remove old step
            bvals[i] += 0.5*leps

            # change the difference by a factor of two
            cureps *= epsscale
            if cureps < mineps or flipflop > flipflopmax:
                # if no convergence set flat derivative (TODO: check if there is a better thing to do instead)
                warnings.warn("Derivative calculation did not converge: setting flat derivative.")
                grads[count] = 0.
                break
            leps *= epsscale

            # central difference
            fvals[i] += 0.5*leps  # change forwards distance to half eps
            bvals[i] -= 0.5*leps  # change backwards distance to half eps
            cdiffnew = (func(fvals)-func(bvals))/leps

            if cdiffnew == cdiff:
                grads[count] = cdiff
                break

            # check whether previous diff and current diff are the same within reltol
            rat = (cdiff/cdiffnew)
            if np.isfinite(rat) and rat > 0.:
                # gradient has not changed sign
                if np.abs(1.-rat) < reltol:
                    grads[count] = cdiffnew
                    break
                else:
                    cdiff = cdiffnew
                    continue
            else:
                cdiff = cdiffnew
                flipflop += 1
                continue

        count += 1

    return grads


In [ ]:
# define a theano Op for our likelihood function
class LogLikeWithGrad(tt.Op):

    itypes = [tt.dvector] # expects a vector of parameter values when called
    otypes = [tt.dscalar] # outputs a single scalar value (the log likelihood)

    def __init__(self, loglike, data):
        """
        Initialise with various things that the function requires. Below
        are the things that are needed in this particular example.

        Parameters
        ----------
        loglike:
            The log-likelihood (or whatever) function we've defined
        data:
            The "observed" data that our log-likelihood function takes in
        """

        # add inputs as class attributes
        self.likelihood = loglike
        self.data = data

        # initialise the gradient Op (below)
        self.logpgrad = LogLikeGrad(self.likelihood, self.data)

    def perform(self, node, inputs, outputs):
        # the method that is used when calling the Op
        theta, = inputs  # this will contain my variables

        # call the log-likelihood function
        logl = self.likelihood(theta, self.data)

        outputs[0][0] = np.array(logl) # output the log-likelihood

    def grad(self, inputs, g):
        # the method that calculates the gradients - it actually returns the
        # vector-Jacobian product - g[0] is a vector of parameter values
        theta, = inputs  # our parameters
        return [g[0]*self.logpgrad(theta)]


class LogLikeGrad(tt.Op):

    """
    This Op will be called with a vector of values and also return a vector of
    values - the gradients in each dimension.
    """
    itypes = [tt.dvector]
    otypes = [tt.dvector]

    def __init__(self, loglike, data):
        """
        Initialise with various things that the function requires. Below
        are the things that are needed in this particular example.

        Parameters
        ----------
        loglike:
            The log-likelihood (or whatever) function we've defined
        data:
            The "observed" data that our log-likelihood function takes in
        """

        # add inputs as class attributes
        self.likelihood = loglike
        self.data = data

    def perform(self, node, inputs, outputs):
        theta, = inputs

        # define version of likelihood function to pass to derivative function
        def lnlike(values):
            return self.likelihood(values, self.data)

        # calculate gradients
        grads = gradients(theta, lnlike)

        outputs[0][0] = grads

In [ ]:
# Run the MCMC using NUTS
# create our Op
my_loglike = cl_lkl
logl = LogLikeWithGrad(my_loglike, data)

ndraws = 100
nburn = 10
# use PyMC3 to sampler from log-likelihood
with pm.Model() as opmodel:
    # uniform priors on m and c
    h = pm.Uniform('h', lower=0.6, upper=0.7)
    Omega_c = pm.Uniform('Omega_c', lower=0.22, upper=0.30)

    # convert m and c to a tensor vector
    theta = tt.as_tensor_variable([h, Omega_c])

    # use a DensityDist
    pm.DensityDist('likelihood', lambda v: logl(v), observed={'v': theta})

    trace = pm.sample(ndraws, tune=nburn, discard_tuned_samples=True)

# plot the traces
_ = pm.traceplot(trace, lines={'h': theta_fid[0], 'Omega_c': theta_fid[1]})

# put the chains in an array (for later!)
# samples_pymc3_2 = np.vstack((trace['m'], trace['c'])).T

In [ ]:
# Run the MCMC using Metropolis-Hasting
# create our Op
my_loglike = cl_lkl
logl = LogLikeWithGrad(my_loglike, data)

ndraws = 100
nburn = 10
# use PyMC3 to sampler from log-likelihood
with pm.Model() as opmodel:
    # uniform priors on m and c
    h = pm.Uniform('h', lower=0.6, upper=0.7)
    Omega_c = pm.Uniform('Omega_c', lower=0.22, upper=0.30)

    # convert m and c to a tensor vector
    theta = tt.as_tensor_variable([h, Omega_c])

    # use a DensityDist
    pm.DensityDist('likelihood', lambda v: logl(v), observed={'v': theta})

    step = pm.Metropolis()
    trace = pm.sample(ndraws, step=step, tune=nburn, discard_tuned_samples=True)

# plot the traces
_ = pm.traceplot(trace, lines={'h': theta_fid[0], 'Omega_c': theta_fid[1]})

# put the chains in an array (for later!)
# samples_pymc3_2 = np.vstack((trace['m'], trace['c'])).T

# Example: Gaussian process

In [ ]:
# set the seed
np.random.seed(1)

n = 100 # The number of data points
X = np.linspace(0, 10, n)[:, None] # The inputs to the GP, they must be arranged as a column vector

# Define the true covariance function and its parameters
ℓ_true = 1.0
η_true = 3.0
cov_func = η_true**2 * pm.gp.cov.Matern52(1, ℓ_true)

# A mean function that is zero everywhere
mean_func = pm.gp.mean.Zero()

# The latent function values are one sample from a multivariate normal
# Note that we have to call `eval()` because PyMC3 built on top of Theano
f_true = np.random.multivariate_normal(mean_func(X).eval(),
                                       cov_func(X).eval() + 1e-8*np.eye(n), 1).flatten()

# The observed data is the latent function plus a small amount of T distributed noise
# The standard deviation of the noise is `sigma`, and the degrees of freedom is `nu`
σ_true = 2.0
ν_true = 3.0
y = f_true + σ_true * np.random.standard_t(ν_true, size=n)

## Plot the data and the unobserved latent function
fig = plt.figure(figsize=(12,5)); ax = fig.gca()
ax.plot(X, f_true, "dodgerblue", lw=3, label="True f");
ax.plot(X, y, 'ok', ms=3, label="Data");
ax.set_xlabel("X"); ax.set_ylabel("y"); plt.legend();

In [ ]:
print('X shape is:', X.shape)
print('f shape is:', f_true.shape)
print('y shape is:', y.shape)
print('cov shape is:', cov_func(X).eval().shape)

In [ ]:
with pm.Model() as model:
    ℓ = pm.Gamma("ℓ", alpha=2, beta=1)
    η = pm.HalfCauchy("η", beta=5)

    cov = η**2 * pm.gp.cov.Matern52(1, ℓ)
    gp = pm.gp.Latent(cov_func=cov)

    f = gp.prior("f", X=X)

    σ = pm.HalfCauchy("σ", beta=5)
    ν = pm.Gamma("ν", alpha=2, beta=0.1)
    j = pm.StudentT("y", mu=f, lam=1.0/σ, nu=ν, observed=y)

    trace = pm.sample(100, chains=1)

In [ ]:
trace.varnames

In [ ]:
# plot the results
fig = plt.figure(figsize=(12,5)); ax = fig.gca()

# plot the samples from the gp posterior with samples and shading
from pymc3.gp.util import plot_gp_dist
plot_gp_dist(ax, trace["f"], X);


# plot the data and the true latent function
plt.plot(X, f_true, "dodgerblue", lw=3, label="True f");
plt.plot(X, y, 'ok', ms=3, alpha=0.5, label="Observed data");

# axis labels and title
plt.xlabel("X"); plt.ylabel("True f(x)");
plt.title("Posterior distribution over $f(x)$ at the observed values"); plt.legend();

plt.ylim([-10, 10])